<a href="https://colab.research.google.com/github/uervitonsantos/Avaliador_de_Acoes_com_base_em_Fundamentos/blob/main/Dados_Fundamentalistas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Avaliador de Ações com base em Fundamentos**


Este script Python realiza uma análise fundamentalista de ações da B3 utilizando os dados do site Fundamentus. O objetivo é pontuar e ranquear ações com base em critérios quantitativos de qualidade e valuation.

⚙️ Funcionalidades
Conecta-se automaticamente ao Fundamentus e carrega os dados financeiros das empresas listadas na bolsa.

Aplica critérios básicos e avançados para avaliar a qualidade da empresa, tais como:

Dividend Yield

Preço/Lucro (P/L)

ROE, ROIC, Margem Líquida

Endividamento, Liquidez, Crescimento de Receita

Para cada critério atendido, a empresa recebe 1 ponto.

Ao final, exibe a pontuação de cada empresa e gera um ranking comparativo.

⚠️ Penalidade
Se uma ação apresentar Dividend Yield abaixo de 6%, ela recebe uma penalização automática, tendo sua pontuação total reduzida em 50%.

In [25]:
%%capture
!pip install fundamentus

In [26]:
import fundamentus
import pandas as pd
import time
import os

In [27]:
# --- Caminho do OneDrive (ajuste se necessário) ---
caminho_onedrive = os.path.expanduser('~/OneDrive/Documentos')  # ou outro caminho como 'C:/Users/SeuUsuario/OneDrive/...'
nome_arquivo = 'ranking_final.csv'
caminho_completo = os.path.join(caminho_onedrive, nome_arquivo)

In [28]:
# --- Definição dos critérios e regras (pontuação 1 ponto por critério atendido) ---
CRITERIOS_BASICOS = {
    'Div.Yield': lambda x: x >= 0.06,                # Preferência por empresas com bons dividendos (>6%)
    'P/L': lambda x: 0 < x <= 15,                    # Evita empresas muito caras em relação ao lucro
    'ROE': lambda x: x >= 0.10,                      # Retorno sobre patrimônio líquido decente
    'Dív.Brut/ Patrim.': lambda x: x < 1,            # Empresas pouco endividadas
    'Cresc. Rec.5a': lambda x: x >= 0.05             # Crescimento estável de receitas
}

In [29]:
# --- Critérios avançados ---
CRITERIOS_AVANCADOS = {
    'P/VP': lambda x: x < 2,                         # Preço/Valor Patrimonial baixo
    'EV/EBITDA': lambda x: x < 10,                   # Valuation ajustado pela dívida
    'Mrg. Líq.': lambda x: x > 0.10,                 # Margem líquida acima de 10%
    'ROIC': lambda x: x > 0.10,                      # Retorno sobre capital investido
    'Liq. Corr.': lambda x: x > 1.5,                 # Liquidez corrente acima de 1.5
    'Liq.2meses': lambda x: x > 1000000              # Liquidez de mercado razoável (> R$1M)
}

In [30]:
# --- Setores priorizados (com base nos códigos 1, 2, 3, 5, 32, 33, 35, 38, 42) ---
SETORES_PRIORIZADOS = {
    'Agropecuária',
    'Água e Saneamento',
    'Alimentos Processados',
    'Material de Transporte',
    'Energia Elétrica',
    'Máquinas e Equipamentos',
    'Previdência e Seguros',
    'Automóveis e Motocicletas'
    'Químicos',
    'Construção Civil',
    'Telecomunicações',
    'Serv.Méd.Hospit. Análises e Diagnósticos',
    'Tecidos, Vestuário e Calçados',
    'Mineração',
    'Madeira e Papel',
    'Intermediários Financeiros',
    'Petróleo, Gás e Biocombustíveis',
}

In [31]:
# --- Combina os dois grupos de critérios ---
CRITERIOS = {**CRITERIOS_BASICOS, **CRITERIOS_AVANCADOS}

In [32]:
# --- Carrega os dados do Fundamentus ---
def carregar_dados_fundamentus():
    print("⏳ Carregando dados do Fundamentus...")
    df = fundamentus.get_resultado_raw()
    print("✅ Dados carregados com sucesso!")
    print(df.columns)
    return df

In [33]:
# --- Formata valores numéricos e percentuais ---
def formatar_valor(valor, percentual=False):
    if percentual:
        return f"{valor * 100:.1f}%"
    elif valor >= 1000:
        return f"R${valor:,.0f}".replace(",", ".")
    return f"{valor:.2f}"

In [34]:
# --- Avalia uma ação com base nos critérios definidos ---
def avaliar_acao(ticker, df):
    print(f"\n🔍 Analisando {ticker}...")

    if ticker not in df.index:
        print(f"⚠️  {ticker} não encontrada nos dados.")
        return 0

    dados_acao = df.loc[ticker]
    pontuacao = 0
    total_criterios = len(CRITERIOS)

    for criterio, regra in CRITERIOS.items():
        try:
            valor = dados_acao[criterio]

            if pd.isna(valor):
                print(f"  ⚠️  '{criterio}' é NaN. Ignorando.")
                continue

            # Define quais critérios são percentuais
            criterios_percentuais = ['Div.Yield', 'ROE', 'Cresc. Rec.5a', 'Mrg. Líq.', 'ROIC']
            valor_formatado = formatar_valor(valor, percentual=criterio in criterios_percentuais)

            if regra(valor):
                pontuacao += 1
                print(f"  ✅ {criterio}: {valor_formatado} (Critério atendido)")
            else:
                print(f"  ❌ {criterio}: {valor_formatado} (Critério não atendido)")

        except KeyError:
            print(f"  ⚠️  Coluna '{criterio}' não encontrada.")
        except Exception as e:
            print(f"  ❌ Erro ao processar '{criterio}': {e}")

    # # --- Verifica setor priorizado ---
    try:
        df_papel = fundamentus.get_papel(ticker)
        setor = df_papel.loc[ticker, 'Setor']

        if setor in SETORES_PRIORIZADOS:
            pontuacao += 1
            print(f"  ⭐ Setor '{setor}' é priorizado! (+1 ponto extra)")
        else:
            print(f"  ℹ️  Setor '{setor}' não está entre os priorizados.")

    except Exception as e:
        print(f"  ⚠️  Erro ao verificar setor de {ticker}: {e}")

    # --- Penalização por Div.Yield baixo (<6%) ---
    try:
        dy = dados_acao['Div.Yield']
        if not pd.isna(dy) and dy < 0.06:
            pontuacao_original = pontuacao
            pontuacao = max(0, pontuacao * 0.5)
            print(f"  ⚠️ Dividend Yield abaixo de 6%: {dy:.2%} → Penalização aplicada (50% da pontuação)")
            print(f"     Pontuação ajustada de {pontuacao_original:.1f} para {pontuacao:.1f}")
    except Exception as e:
        print(f"  ⚠️ Erro ao verificar Div.Yield para penalização: {e}")

    percentual_total = (pontuacao / (total_criterios + 1)) * 100
    print(f"📊 Pontuação final para {ticker}: {pontuacao:.1f}/{total_criterios + 1} ({percentual_total:.2f}%)")
    return pontuacao

In [35]:
# --- Execução principal ---
def executar_avaliacao(lista_acoes):
    df = carregar_dados_fundamentus()
    resultados = []

    for acao in lista_acoes:
        pontuacao = avaliar_acao(acao, df)
        resultados.append((acao, pontuacao))

    print("\n📋 Ranking Final:")
    df_resultado = pd.DataFrame(resultados, columns=["Ação", "Pontuação"]).sort_values(by="Pontuação", ascending=False)
    print(df_resultado)

    # --- Salvando CSV ---
    try:
        df_resultado.to_csv(caminho_completo, index=False, sep=';', encoding='utf-8-sig')
        print(f"\n💾 Arquivo CSV salvo com sucesso em: {caminho_completo}")
    except Exception as e:
        print(f"❌ Erro ao salvar o arquivo CSV: {e}")

    return df_resultado

In [36]:
# --- Executar se for o módulo principal ---
if __name__ == "__main__":
    acoes = ['OIBR3', 'ITUB4', 'PETR4', 'BBAS3', 'MGLU3']
    executar_avaliacao(acoes)

⏳ Carregando dados do Fundamentus...


/usr/local/lib/python3.11/dist-packages/fundamentus/resultado.py:50: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(content.text, decimal=",", thousands='.')[0]


✅ Dados carregados com sucesso!
Index(['Cotação', 'P/L', 'P/VP', 'PSR', 'Div.Yield', 'P/Ativo', 'P/Cap.Giro',
       'P/EBIT', 'P/Ativ Circ.Liq', 'EV/EBIT', 'EV/EBITDA', 'Mrg Ebit',
       'Mrg. Líq.', 'Liq. Corr.', 'ROIC', 'ROE', 'Liq.2meses', 'Patrim. Líq',
       'Dív.Brut/ Patrim.', 'Cresc. Rec.5a'],
      dtype='object', name='Multiples')

🔍 Analisando OIBR3...
  ❌ Div.Yield: 0.0% (Critério não atendido)
  ✅ P/L: 0.02 (Critério atendido)
  ❌ ROE: -95.9% (Critério não atendido)
  ✅ Dív.Brut/ Patrim.: -0.77 (Critério atendido)
  ❌ Cresc. Rec.5a: -25.5% (Critério não atendido)
  ✅ P/VP: -0.02 (Critério atendido)
  ✅ EV/EBITDA: -4.88 (Critério atendido)
  ✅ Mrg. Líq.: 847.4% (Critério atendido)
  ❌ ROIC: -18.2% (Critério não atendido)
  ❌ Liq. Corr.: 0.70 (Critério não atendido)
  ✅ Liq.2meses: R$4.268.500 (Critério atendido)
  ⭐ Setor 'Telecomunicações' é priorizado! (+1 ponto extra)
  ⚠️ Dividend Yield abaixo de 6%: 0.00% → Penalização aplicada (50% da pontuação)
     Pontuação ajus

/usr/local/lib/python3.11/dist-packages/fundamentus/detalhes.py:232: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables_html = pd.read_html(content.text, decimal=",", thousands='.')
/usr/local/lib/python3.11/dist-packages/fundamentus/detalhes.py:232: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables_html = pd.read_html(content.text, decimal=",", thousands='.')
/usr/local/lib/python3.11/dist-packages/fundamentus/detalhes.py:232: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables_html = pd.read_html(content.text, decimal=",", thousands='.')


  ⭐ Setor 'Intermediários Financeiros' é priorizado! (+1 ponto extra)
📊 Pontuação final para BBAS3: 9.0/12 (75.00%)

🔍 Analisando MGLU3...
  ❌ Div.Yield: 3.2% (Critério não atendido)
  ❌ P/L: 16.40 (Critério não atendido)
  ❌ ROE: 3.8% (Critério não atendido)
  ✅ Dív.Brut/ Patrim.: 0.40 (Critério atendido)
  ❌ Cresc. Rec.5a: 3.8% (Critério não atendido)
  ✅ P/VP: 0.63 (Critério atendido)
  ✅ EV/EBITDA: 3.17 (Critério atendido)
  ❌ Mrg. Líq.: 1.1% (Critério não atendido)
  ❌ ROIC: 7.2% (Critério não atendido)
  ❌ Liq. Corr.: 1.20 (Critério não atendido)
  ✅ Liq.2meses: R$283.870.000 (Critério atendido)
  ℹ️  Setor 'Comércio' não está entre os priorizados.
  ⚠️ Dividend Yield abaixo de 6%: 3.17% → Penalização aplicada (50% da pontuação)
     Pontuação ajustada de 4.0 para 2.0
📊 Pontuação final para MGLU3: 2.0/12 (16.67%)

📋 Ranking Final:
    Ação  Pontuação
2  PETR4       10.0
3  BBAS3        9.0
1  ITUB4        8.0
0  OIBR3        3.5
4  MGLU3        2.0
❌ Erro ao salvar o arquivo CSV:

/usr/local/lib/python3.11/dist-packages/fundamentus/detalhes.py:232: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables_html = pd.read_html(content.text, decimal=",", thousands='.')
/usr/local/lib/python3.11/dist-packages/fundamentus/detalhes.py:232: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables_html = pd.read_html(content.text, decimal=",", thousands='.')
